# PLOT PROBABILITY DENSITIES

This notebook can be used to generate the probability densities surface plots from the files provided by QSLE-v1.0 program. In addition, the user can modify the units of measure, for example changing the radians into degrees.

Please read this notebook carefully and follow the procedure step by step.

## Import Libraries

To use this notebook, the following libraries need to be imported:
* `Numpy`
* `Scipy`
* `Pandas`
* `Matplotlib`
* `Tkinter`

After having installed the mentioned libraries, you can run the following cells.

In [ ]:
import numpy as np
import scipy.constants as sc
import pandas as pd
import tkinter as tk
from tkinter import filedialog
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.cm import ScalarMappable
import matplotlib.colors as mcol
import matplotlib.cm as cm
from matplotlib.ticker import AutoMinorLocator

def error_exit():
    print("\n### Terminating program because of errors ###\n")
    exit(1)

print("\n********************************")
print("* LIBRARIES LOADED SUCCESSFULLY*")
print("********************************")

## Choosing The Restart File

The first thing you must do is to select the restart file printed from the QSLE run (i.e. `PREFIX_restart.inp`).

Run the following cell and choose the right *.inp filename.

In [ ]:
root=tk.Tk()
root.withdraw()
restart= filedialog.askopenfilename(title="CHOOSE THE RESTART FILE", filetypes=(("restart files","*.inp"),("all files", "*.*")))

input_file=pd.read_csv(restart, sep='\s+', header=None, usecols=[0,1])

angle_points= int(input_file[input_file[0].str.contains("Angle_points")].iat[0,1])
mom_points= int(input_file[input_file[0].str.contains("Momentum_points")].iat[0,1])
mom_max= float(input_file[input_file[0].str.contains("Mom_max")].iat[0,1])
prefix= input_file[input_file[0].str.contains("Prefix")].iat[0,1]

mom_grid=np.linspace(-mom_max,mom_max,mom_points)
angle_grid=np.zeros(angle_points)
for i in range(angle_points):
    angle_grid[i]=-np.pi+i*2.*np.pi/angle_points

print("\n*************************************")
print("* RESTART FILE IMPORTED SUCCESSFULLY*")
print("*************************************")

## Choosing The Units Of Measure

Now, you can set the units of measure you want to adopt. 

Run the cell below and follow the instructions on the appearing windows.

In [ ]:
list_titles=["ANGLE", "LENGTH", "TIME", "MASS"]
list_selections=["RADIAN", "ANGSTROM", "FEMTOSECOND", "DALTON"]
sel_units = ["rad", "Å", "fs", "Da"]
list_choices=[["RADIAN", "DEGREE"],
              ["ANGSTROM", "MICROMETER", "NANOMETER", "PICOMETER", "FEMTOMETER", "AU_LENGTH", "SI_LENGTH", "CGS_LENGTH"],
              ["FEMTOSECOND", "NANOSECOND", "PICOSECOND","ATTOSECOND", "AU_TIME", "SI_TIME", "CGS_TIME"],
              ["DALTON", "AU_MASS", "SI_MASS", "CGS_MASS"]]
list_units=[["rad", "deg"],
              ["Å", "μm", "nm", "pm", "fm", "a0", "m", "cm"],
              ["fs", "ns", "ps","as", "(1/(4πcR))", "s", "s"],
              ["Da", "me", "kg", "g"]]

list_coeff=[[1.0, 180.0/sc.pi],
              [1.0, 10**(-4), 10**(-1), 10**(2), 10**(5), 10**(-10)/(sc.Planck**2/(4 * sc.pi**2 * sc.m_e * sc.e**2)), 10**(-10), 10**(-8)],
              [1.0, 10**(-6), 10**(-3), 10**(3), 10**(-15)*(4 * sc.pi * sc.c * sc.Rydberg), 10**(-15), 10**(-15)],
              [1.0,  sc.atomic_mass /sc.m_e, sc.atomic_mass, sc.atomic_mass*1000]]


print("\n********************************")
print("* STARTING THE CHOICE OF UNITS *")
print("********************************")


for i in range(len(list_selections)):
    appear = True
    def showSelected():
        global appear
        global list_selections
        global i
        show.config(text=str(bigListbox.get("anchor"))+ " is your current choice.\nIf you are sure, press NEXT.")
        list_selections[i]=str(bigListbox.get("anchor"))
        if appear == True:
            show.pack()
            b1.pack(pady=10)
            appear = False
    def procede():
        rootWindow.quit()
        rootWindow.destroy()

    bigArr=list_choices[i]

    rootWindow = tk.Tk()
    rootWindow.title(list_titles[i] + " UNIT")
    height = 150+25*len(bigArr)
    rootWindow.geometry("300x" +str(height))

    bigListbox = tk.Listbox(rootWindow, height=len(bigArr), selectmode= "single", font=10)
    bigListbox.pack(pady=10)

    myIndex = len(bigArr)
    for index in range(0, myIndex):
        bigListbox.insert(myIndex, bigArr[index])

    tk.Button(rootWindow, text='SELECT HIGHLIGHTED KEYWORD', command=showSelected).pack(pady=10)

    show = tk.Label(rootWindow)
    show.config(text= list_selections[i] +" is your current choice.\nIf you are sure, press NEXT.")
    b1 = tk.Button(rootWindow, text='NEXT', command=procede)

    rootWindow.mainloop()

    print("+ {0} --> {1}".format(list_titles[i],list_selections[i]))

angle_converter= 1.0
if list_selections[0]=="DEGREE":
    angle_converter = 180.0/sc.pi

mom_converter = 1.0
for i in range(len(list_selections)):
    choices = list_choices[i]
    coeff = list_coeff[i]
    units = list_units[i]
    for ii in range(len(choices)):
        if list_selections[i] == choices[ii]:
            if list_titles[i] == "ANGLE":
                mom_converter *= coeff[ii]
                sel_units[i] = units[ii]
            if list_titles[i] == "LENGTH":
                mom_converter *= (coeff[ii]**2)
                sel_units[i] = units[ii]
            if list_titles[i] == "TIME":
                mom_converter /= coeff[ii]
                sel_units[i] = units[ii]
            if list_titles[i] == "MASS":
                mom_converter *= coeff[ii]
                sel_units[i] = units[ii]
            break

time_converter = 1.0
time_choices = list_choices[2]
time_coeff = list_coeff[2]
for i in range(len(time_choices)):
    if list_selections[2] == time_choices[i]:
        time_converter = time_coeff[i]
        break

print("\n*******************************")
print("* UNITS CONVERTED SUCCESSFULLY*")
print("*******************************")

## Choosing The 'Transition Rates' Files

Finally, you can select the output files related to the transition rates (i.e. `PREFIX_m01.dat` or `PREFIX_m10.dat`). 

Run the cell below and choose the *.dat files of your choice (for multiple selection remember to hold down CTRL).

In [ ]:
files= filedialog.askopenfilenames(title="CHOOSE THE m01 or m10 FILES", filetypes=(("output files", prefix + "_m*"),("all files", "*.*")))
directory = files[0][:files[0].rfind('/')]
initial_path= directory + "/Plot_"

A, M = np.meshgrid(angle_grid * angle_converter, mom_grid * mom_converter)

print("\n*********************************")
print("* PLOTTING THE TRANSITION RATES *")
print("*********************************")

for file in files:
    data=np.genfromtxt(file, dtype=np.float32, usecols= [2])
    data /= time_converter
    name= file[file.rfind(prefix):len(file)-4]
    m_choice = file[file.rfind(prefix)+len(prefix) + 1:file.rfind(prefix) +len(prefix)+4]

    params={'figure.figsize':(3.0,5.0),'axes.labelsize':13,'axes.titlesize':15,'xtick.labelsize':10.5,'ytick.labelsize':10.5,'axes.linewidth':'1.',
            'xtick.major.size':'3','xtick.major.width':'1.5','ytick.major.size':'3','ytick.major.width':'1.5',
            'xtick.minor.size':'2','xtick.minor.width':'1','ytick.minor.size':'2','ytick.minor.width':'1',
            'lines.linewidth':'1.5'}
    plt.rcParams.update(params)
    fig,axs=plt.subplots()

    Z = data.reshape(len(angle_grid),len(mom_grid))
    Z=Z.T

    vmax=np.max(data)
    vmin=0

    cbar=axs.contourf(A, M, Z, 1000, cmap='viridis_r', vmin=vmin,vmax=vmax)
    axs.set_xlabel(r"$φ\,/\,$" + sel_units[0])
    axs.set_ylabel(r"$p_φ\,/\,$(" + sel_units[0] +r"$\,$"+ sel_units[1] + r"$^2$" + sel_units[3] +r"$\,$" + sel_units[2] +r"$^{-1}$)" )
    #axs.set_title(m_choice + " / " + sel_units[2] + r"$^{-1}$")
    #axs.yaxis.set_minor_locator(AutoMinorLocator())
    #axs.xaxis.set_minor_locator(AutoMinorLocator())
    axs.ticklabel_format(axis="y", scilimits=[0,0], useMathText = True)
    t2= axs.yaxis.get_offset_text()
    t2.set_x(-0.05)
    if sel_units[0] == "deg":
        axs.set_xticks([-180,-90, 0, 90, 180])
    else:
        axs.set_xticks([-np.pi,-np.pi/2, 0, np.pi/2 , np.pi])
        axs.set_xticklabels(["-π", "-π/2", "0", "π/2", "π"])
    axs.set_aspect(1.0/axs.get_data_ratio(), adjustable='box')

    cbformat = matplotlib.ticker.ScalarFormatter(useMathText = True)
    cbformat.set_powerlimits((0,0))
    plt.colorbar(ScalarMappable(norm=cbar.norm, cmap=cbar.cmap),fraction=0.046, pad=0.15, ax=axs, format = cbformat, orientation = "horizontal")
    plt.savefig(initial_path + name+ ".png",bbox_inches='tight', pad_inches=0.03, dpi=300)
    print("+ {0}.dat --> Plot_{0}.png".format(name))


print("\n****************************************")
print("* TRANSITION RATES PLOTTED SUCCESSFULLY*")
print("****************************************")
